In [1]:
# !python -m spacy download es_core_news_sm

In [1]:
import os
import re
import spacy
import awswrangler as wr
import pandas as pd

## Carga de datos

In [2]:
df_hogar_mania = wr.s3.read_parquet(path='s3://raw-recipe-data-bucket/hogar_mania')
df_antena3 = wr.s3.read_parquet(path='s3://raw-recipe-data-bucket/antena3')
df_saber_vivir = wr.s3.read_parquet(path='s3://raw-recipe-data-bucket/saber_vivir')
df_el_mundo = wr.s3.read_parquet(path='s3://raw-recipe-data-bucket/el_mundo')

### Hogar Mania

In [3]:
categorias_hogar_mania = set(df_hogar_mania['categoria'].unique())
categorias_hogar_mania

{'aperitivos',
 'arroces-cereales',
 'bebidas',
 'carnes',
 'cocina-internacional',
 'cocteles',
 'desayunos',
 'ensaladas-verduras',
 'huevos',
 'legumbres',
 'pastas-pizzas',
 'pescados-mariscos',
 'postres',
 'recetas-pan',
 'salsas',
 'setas-hongos',
 'sopas-cremas'}

Guardamos todas las categorías porque nos harán falta más tarde para unificarlas con el resto de conjunto de datos

In [4]:
df_hogar_mania.shape

(7464, 5)

Eliminamos los recetas con ingredientes nulos porque no son una cantidad representativa, ni tampoco tenemos manera de imputar sus valores

In [4]:
df_hogar_mania.dropna(subset=["ingredientes"], inplace=True)

In [5]:
df_hogar_mania.shape

(7228, 5)

In [6]:
df_hogar_mania['total_ingredientes'] = df_hogar_mania['ingredientes'].apply(len)
df_hogar_mania[df_hogar_mania['total_ingredientes'] == 0].shape

(18, 6)

In [7]:
df_hogar_mania = df_hogar_mania[df_hogar_mania['total_ingredientes'] != 0]

Inspeccionamos las recetas con múltiple categoría para ver si hay alguna manera de elegir con cual quedarnos

In [9]:
df_hogar_mania[df_hogar_mania.duplicated(subset=['link'], keep=False)]['categoria'].unique()

<StringArray>
[             'postres',           'aperitivos',   'ensaladas-verduras',
     'arroces-cereales',        'pastas-pizzas',         'sopas-cremas',
            'legumbres',               'carnes',    'pescados-mariscos',
               'huevos',            'desayunos',          'recetas-pan',
              'bebidas',             'cocteles', 'cocina-internacional']
Length: 15, dtype: string

In [10]:
df_hogar_mania_double = df_hogar_mania[df_hogar_mania.duplicated(subset=['link'], keep=False)]
df_hogar_mania_double.groupby(by='link')['categoria'].apply(list)

link
https://www.hogarmania.com//cocina/recetas/aperitivos/desayuno-real-cereal-37095.html                                        [aperitivos, desayunos]
https://www.hogarmania.com//cocina/recetas/aperitivos/jocoque-10518.html                                          [aperitivos, cocina-internacional]
https://www.hogarmania.com//cocina/recetas/aperitivos/molletes-mexicana-36967.html                                           [aperitivos, desayunos]
https://www.hogarmania.com//cocina/recetas/aperitivos/zumo-papaya-naranja-pomelo-25672.html                                    [aperitivos, bebidas]
https://www.hogarmania.com//cocina/recetas/arroces-cereales/arroz-colores-salsa-agridulce-asia-2684.html    [arroces-cereales, cocina-internacional]
                                                                                                                              ...                   
https://www.hogarmania.com//cocina/recetas/recetas-pan/bagels-rosquillas-18773.html                  

Vemos que no hay un criterio que podamos seguir, así que nos decantamos por quedarnos con la primera categoría que se encuentre en el conjunto de datos

In [8]:
df_hogar_mania.drop_duplicates(subset=['link'], keep='first', inplace=True)

In [9]:
df_hogar_mania.shape

(7056, 6)

In [10]:
def extract_words(text):
    words = re.findall(r'\b[a-zA-Z]+\b', text)
    return ' '.join(words)

In [11]:
df_hogar_mania['titulo_link'] = (
    df_hogar_mania['link']
    .apply(os.path.basename)
    .apply(os.path.splitext)
    .apply(lambda text: text[0])
    .apply(extract_words)
)
df_hogar_mania[['titulo', 'titulo_link']]

,titulo,titulo_link
0,"Buñuelos de viento fáciles, la receta tradicio...",bunuelos
1,Corona de hojaldre de Navidad,corona navidad
2,Cafés de Navidad: Gingerbread Latte y Pumpkin ...,cafes navidad gingerbread pumpkin spice latte
4,Mazapanes de Navidad,mazapanes navidad
6,"Churros en freidora de aire, ¡más fácil imposi...",churros freidora aire
...,...,...
7384,"Tarta fría de limón de Eva Arguiñano, la más c...",tarta fria limon
7385,Receta de mochis japoneses caseros,receta mochis japoneses caseros
7386,"Receta de helado de pistacho, paso a paso",helado pistacho casero
7387,Cómo hacer gachas dulces con los pasos de Karl...,gachas dulces


Construimos una nueva columna dado que los títulos tienen bastante ruido, en cambio si nos fijamos en el link los titulos son mas concisos y directos

In [12]:
nlp = spacy.load('es_core_news_sm')

In [14]:
def extract_ingredients(ingrs):
    unnecessary_bow = [
        "gr",
        "g",
        "kg",
        "ml",
        "l",
        "oz",
        "lb",
        "gr.",
        "g.",
        "kg.",
        "ml.",
        "l.",
        "oz.",
        "lb.",
        "cm",
        "kilo",
        "litro",
        "gramo",
        "gramos",
        "vaso",
        "vasos",
        "taza",
        "tazón",
        "pizca",
        "cucharada",
        "cucharadas",
        "cucharadita",
        "ralladura",
        "chorrito"
    ]
    admitted_class_words = ["NOUN", "PROPN", "PRON"] # 220 gr. azúcar | 1 cebolla => lo identifica como VERB ?!
    filtered_tokens = []
    for ing in ingrs: 
        doc = nlp(ing.lower())
        filtered_tokens += [
            token.text
            for token in doc
            if not token.like_num
            and not token.is_stop
            and not token.is_punct
            and token.pos_ in admitted_class_words
            and token.text not in unnecessary_bow
        ]
    return " ".join(filtered_tokens)

Construimos una funcion para la tokenización de los ingredientes:
1. Bolsa de palabras "innecesarias" o que no añaden valor, sobre todo relacionadas con medidas en la cocina
2. Eliminación de palabras vacías, números o signos de puntuación
3. Nos quedamos con los tokens que son sustantivos o pronombres

In [15]:
df_hogar_mania['ingredientes_limpios'] = df_hogar_mania['ingredientes'].apply(extract_ingredients)
df_hogar_mania.head(5)

,titulo,categoria,ingredientes,elaboracion,link,total_ingredientes,titulo_link,ingredientes_limpios
0,"Buñuelos de viento fáciles, la receta tradicio...",postres,"[125 gr de harina, 30 gr de mantequilla, 1/4 l...",Otoño no es solo época de calabazas y castañas...,https://www.hogarmania.com//cocina/recetas/pos...,9,bunuelos,harina mantequilla agua azúcar huevos limón ac...
1,Corona de hojaldre de Navidad,postres,"[2 láminas de hojaldre rectangular , 150 g de ...","Los polvorones, los turrones, los mazapanes o ...",https://www.hogarmania.com//cocina/recetas/pos...,7,corona navidad,láminas hojaldre chocolate mantequilla glas mo...
2,Cafés de Navidad: Gingerbread Latte y Pumpkin ...,postres,"[2 cucharadas de azúcar moreno suave, 1/2 cuch...","La temporada de invierno, junto a la época nav...",https://www.hogarmania.com//cocina/recetas/pos...,8,cafes navidad gingerbread pumpkin spice latte,azúcar jengibre nuez moscada canela leche café...
4,Mazapanes de Navidad,postres,"[300 gr. de almendra molida , 370 gr. de leche...",Mezcla en un bol la almendra molida con la lec...,https://www.hogarmania.com//cocina/recetas/pos...,7,mazapanes navidad,almendra leche limón agua aceite oliva virgen ...
6,"Churros en freidora de aire, ¡más fácil imposi...",postres,"[Churros congelados, Aceite de oliva o girasol...",Comienza por precalentar la freidora de aire. ...,https://www.hogarmania.com//cocina/recetas/pos...,3,churros freidora aire,churros aceite oliva girasol azúcar


### Antena3

In [17]:
categorias_antena3 = set(df_antena3['categoria'].unique())
categorias_antena3

{'arroces',
 'carnes',
 'ensaladas',
 'entrantes',
 'guisos',
 'huevos',
 'pasta',
 'pescado',
 'postres'}

Volvemos a quedarnos con las categorías para más tarde

In [18]:
df_antena3.shape

(2022, 5)

In [19]:
df_antena3['total_ingredientes'] = df_antena3['ingredientes'].apply(len)

Eliminamos las recetas sin ingredientes o elaboracion

In [20]:
df_antena3 = df_antena3[df_antena3['total_ingredientes'] != 0]
df_antena3 = df_antena3[(df_antena3['elaboracion'].str.len() != 0)]

In [21]:
df_antena3.shape

(1761, 6)

In [22]:
df_antena3[df_antena3['total_ingredientes'] > 14].shape

(428, 6)

Recortamos las recetas con más de 14 ingredientes debido al ruido que se observo en la exploración

In [23]:
df_antena3['ingredientes'] = (
    df_antena3
    ['ingredientes']
    .apply(lambda ingrs: ingrs[:14] if len(ingrs) > 14 else ingrs)
)

In [24]:
df_antena3['total_ingredientes'] = df_antena3['ingredientes'].apply(len)
df_antena3['total_ingredientes'].describe()

count    1761.000000
mean       10.781374
std         2.838336
min         2.000000
25%         8.000000
50%        11.000000
75%        14.000000
max        14.000000
Name: total_ingredientes, dtype: float64

In [25]:
df_antena3['titulo_link'] = (
    df_antena3['link']
    .apply(os.path.basename)
    .apply(os.path.splitext)
    .apply(lambda text: text[0])
    .apply(extract_words)
)
df_antena3[['titulo', 'titulo_link']]

,titulo,titulo_link
0,"Receta de alcachofas en salsa, de Karlos Argui...",alcachofas
1,"Una receta sencilla, sabrosa y rápida de Joseb...",ensaladilla
2,"Brócoli con champiñones y patatas, de Arguiñan...",brocoli champinones
3,"Berenjenas a la crema, de Karlos Arguiñano: ""U...",berenjenas
4,Espárragos a la plancha con vinagreta de fresa...,esparragos plancha vinagreta fresas
...,...,...
2017,Arroz con verduras y conejo al azafrán,video arroz con verduras y
2018,La deliciosa receta de canelones de cocido de ...,la deliciosa receta de canelones de cocido de ...
2019,Macarrones con crema de calabaza y queso,macarrones crema calabaza
2020,Canelones de verduras y jamón cocido,canelones de verduras y jamon


Misma operacion con los titulos y tambien la tokenización de los ingredientes

In [26]:
df_antena3['ingredientes_limpios'] = df_antena3['ingredientes'].apply(extract_ingredients)
df_antena3[['ingredientes','ingredientes_limpios']]

,ingredientes,ingredientes_limpios
0,"[8 huevos de codorniz, 1 cebolla, 2 dientes de...",huevos dientes ajo txakoli caldo verduras hari...
1,"[3 patatas, 3 zanahorias, 3 huevos, 4 lonchas ...",patatas zanahorias huevos lonchas queso havart...
2,"[6 champiñones, 2 patatas, 4 dientes de ajo, 1...",champiñones patatas dientes ajo jamón lochas f...
3,"[1 cebolla pequeña, 4 dientes de ajo, ½ l de l...",cebolla dientes ajo leche caldo carne gambas j...
4,"[4-6 fresas (dependiendo del tamaño), ½ aguaca...",fresas tamaño aguacate lima aceite oliva virge...
...,...,...
2017,"[300 g de arroz, 2 muslos de conejo, 1 cebolle...",arroz muslos conejo espárragos dientes ajo alc...
2018,"[24 láminas de canelón, 1 muslo de gallina (co...",láminas canelón muslo gallina trozos rabo tern...
2019,"[300 g de macarrones, 400 g de calabaza (potim...",macarrones calabaza potimarrón puerro queso na...
2020,"[20 láminas de canelones, 200 g de espinacas, ...",láminas canelones espinacas zanahorias dientes...


### Saber Vivir

Seguiremos el mismos pasos que antes

In [27]:
categorias_saber_vivir = set(df_saber_vivir['categoria'].unique())
categorias_saber_vivir_low = set([categoria.lower() for categoria in categorias_saber_vivir])
categorias_saber_vivir_low

{'arroces',
 'caldos y cremas',
 'carnes',
 'celebraciones',
 'ensaladas y verduras',
 'legumbres',
 'pastas y pizzas',
 'pasteles salados',
 'pescados y mariscos',
 'postres y dulces'}

In [28]:
df_saber_vivir.shape

(461, 5)

In [29]:
df_saber_vivir['total_ingredientes'] = df_saber_vivir['ingredientes'].apply(len)

In [30]:
df_saber_vivir['titulo_link'] = (
    df_saber_vivir['link']
    .apply(os.path.basename)
    .apply(os.path.splitext)
    .apply(lambda text: text[0])
    .apply(extract_words)
)
df_saber_vivir[['titulo', 'titulo_link']]

,titulo,titulo_link
0,Crema de calabaza,crema de calabaza
1,Crema de champiñones y queso,crema de champinones y queso
2,Crema de coles de Bruselas con jamón,crema de coles de bruselas con jamon
3,Crema de espárragos con caviar,crema de esparragos con caviar
4,Crema de espinacas,crema de espinacas
...,...,...
456,Crema castellana con huevo,crema castellana con huevo
457,Crema catalana a los tres cítricos,crema catalana a los tres citricos
458,Crema catalana con hierbaluisa,crema catalana con hierbaluisa
459,"Crema de boletus, castañas y berenjenas",crema de boletus castanas y berenjenas


In [31]:
df_saber_vivir['ingredientes_limpios'] = df_saber_vivir['ingredientes'].apply(extract_ingredients)
df_saber_vivir[['ingredientes','ingredientes_limpios']]

,ingredientes,ingredientes_limpios
0,"[1 puerro pequeño, Media cebolla, 1 zanahoria,...",puerro cebolla tomate patatas calabaza brick n...
1,"[600 gr. champiñones, 4 vasos leche, 2 vaso de...",champiñones leche ave quesitos patatas setas l...
2,"[12 coles de Bruselas, 5 patatas medianas, Ace...",coles bruselas patatas aceite oliva blanca cal...
3,"[1 manojo de espárragos verdes, 1 litro de cal...",manojo espárragos caldo verduras lata mújol ho...
4,"[2 dientes de ajo picados, 1 vaso de arroz coc...",dientes ajo arroz caldo clavos olor cominos es...
...,...,...
456,"[8 dientes de ajo, 4 huevos, 100 gr. pan, 100 ...",dientes ajo huevos pan chorizo caldo ave aceit...
457,"[1 litro leche, 220 gr. azúcar, 40 gr. harina ...",leche harina maíz yemas huevo lima limón
458,"[2 cucharadas de azúcar, 2 cucharadas de nata,...",azúcar nata mantequilla chocolate fondant
459,"[100 gr. boletus, 100 gr. setas shiitake, 80 g...",boletus setas shiitake castañas brandy litros ...


### El Mundo

In [32]:
categorias_el_mundo = set(df_el_mundo['categoria'].unique())
categorias_el_mundo

{'aperitivos-faciles',
 'arroz',
 'bizcochos',
 'carne',
 'cremas',
 'guisos',
 'legumbres',
 'masas-panes',
 'orientales',
 'pasta',
 'patatas-huevos-tortillas',
 'pescados',
 'pollo',
 'postres',
 'recetas-mariscos',
 'salsas',
 'sanas-faciles',
 'sopas-caldos',
 'vegetarianas',
 'verduras'}

In [33]:
df_el_mundo.shape

(388, 5)

In [34]:
df_el_mundo_double = df_el_mundo[df_el_mundo.duplicated(subset=['link'], keep=False)]
df_el_mundo_double.groupby(by='link')['categoria'].apply(list).head(10)

link
https://recetasdecocina.elmundo.es/2013/12/salpicon-marisco-receta-facil.html                              [aperitivos-faciles, recetas-mariscos]
https://recetasdecocina.elmundo.es/2014/05/arroz-con-champinones-receta-sencilla-eficaz.html                 [arroz, vegetarianas, sanas-faciles]
https://recetasdecocina.elmundo.es/2015/03/besugo-al-horno-receta-facil.html                                            [pescados, sanas-faciles]
https://recetasdecocina.elmundo.es/2015/08/pisto-manchego-receta-facil-casera.html                        [verduras, vegetarianas, sanas-faciles]
https://recetasdecocina.elmundo.es/2016/02/pollo-curry-receta-facil-hindu.html                                        [pollo, guisos, orientales]
https://recetasdecocina.elmundo.es/2016/11/masa-pizza-casera-receta-facil.html                                       [masas-panes, sanas-faciles]
https://recetasdecocina.elmundo.es/2017/05/callos-madrilena-receta-casera.html                                         

In [35]:
df_el_mundo_double.shape

(315, 5)

Eliminamos duplicados y recetas sin elaboración detectadas en la exploración de datos

In [36]:
df_el_mundo = df_el_mundo[(df_el_mundo['elaboracion'].str.len() != 0)]
df_el_mundo = df_el_mundo.drop_duplicates(subset=['link'])

In [37]:
df_el_mundo.shape

(209, 5)

Y al igual que antes realizamos la tokenización de los ingredientes

In [38]:
df_el_mundo['ingredientes_limpios'] = df_el_mundo['ingredientes'].apply(extract_ingredients)
df_el_mundo[['ingredientes','ingredientes_limpios']]

,ingredientes,ingredientes_limpios
0,"[Molde de tarta de 28 cm, 200 gramos de gallet...",molde tarta galletas maría mantequilla nata qu...
1,"[8 huevos, 200 gramos de azúcar, 1 litro de le...",huevos azúcar leche maicena canela corteza lim...
2,"[120 gramos de azúcar, 4 huevos, 400 gramos de...",azúcar huevos bizcochos soletilla café + agua ...
3,"[200 gramos de pasta de arroz, 1 ceboleta, Med...",pasta arroz ceboleta pimiento pechuga pollo sa...
4,"[1 bote de judías pintas cocidas, 200 gramos d...",bote judías pintas pasta pimiento pimentón ver...
...,...,...
314,"[2 rebanadas de pan de molde, 2 cucharadas de ...",rebanadas pan molde mayonesa salsa lata atún a...
319,"[100 gramos de copos de avena, 40 gramos de az...",copos avena azúcar avena harina trigo canela p...
323,"[1 kilo de calamares limpios, 2 cebollas, 1 di...",calamares cebollas ajo vino tomate
363,"[Lechuga de hoja grande tipo iceberg, 1 manzan...",hoja tipo manzana granny ramas apio nueces ará...


## Exportar datos limpios

No existe una intersección de categorías entre los cuatro conjuntos de datos, aunque esto tiene truco. Sabemos que algunas semánticamente son iguales, aunque están representadas de forma diferente

In [39]:
set.intersection(categorias_hogar_mania, categorias_antena3, categorias_saber_vivir_low, categorias_el_mundo)

set()

In [40]:
all_categories = {*categorias_hogar_mania, *categorias_antena3, *categorias_saber_vivir_low, *categorias_el_mundo}
print(len(all_categories))
print(sorted(all_categories))

45
['aperitivos', 'aperitivos-faciles', 'arroces', 'arroces-cereales', 'arroz', 'bebidas', 'bizcochos', 'caldos y cremas', 'carne', 'carnes', 'celebraciones', 'cocina-internacional', 'cocteles', 'cremas', 'desayunos', 'ensaladas', 'ensaladas y verduras', 'ensaladas-verduras', 'entrantes', 'guisos', 'huevos', 'legumbres', 'masas-panes', 'orientales', 'pasta', 'pastas y pizzas', 'pastas-pizzas', 'pasteles salados', 'patatas-huevos-tortillas', 'pescado', 'pescados', 'pescados y mariscos', 'pescados-mariscos', 'pollo', 'postres', 'postres y dulces', 'recetas-mariscos', 'recetas-pan', 'salsas', 'sanas-faciles', 'setas-hongos', 'sopas-caldos', 'sopas-cremas', 'vegetarianas', 'verduras']


Reducimos el número de categorías, dado que no nos va a aportar valor significativo contar con 45

In [41]:
definitive_categories = {
    "arroces",
    "bebidas",
    "carnes",
    "desayunos",
    "entrantes",
    "ensaladas",
    "huevos",
    "pastas-pizzas",
    "panes",
    "pescados-mariscos",
    "postres",
    "sopas-cremas",
    "segundos-platos",
    "otros"
}

Se realizan los *mappings* de las categorías de cada conjunto con las seleccionadas. Algunas son evidentes y otras quizás se imputan con cierta subjetividad

In [42]:
hogar_mania_cat_map = {
    "postres": "postres",
    "aperitivos": "entrantes",
    "arroces-cereales": "arroces",
    "ensaladas-verduras": "ensaladas",
    "pastas-pizzas": "pastas-pizzas",
    "sopas-cremas": "sopas-cremas",
    "legumbres": "segundos-platos", # ? !
    "carnes": "carnes",
    "pescados-mariscos": "pescados-mariscos",
    "setas-hongos": "segundos-platos", # ? !
    "huevos": "huevos",
    "salsas": "entrantes", # ? !
    "desayunos": "desayunos",
    "recetas-pan": "panes",
    "bebidas": "bebidas",
    "cocteles": "bebidas",
    "cocina-internacional": "otros" # ? !
}

In [43]:
antena3_cat_map = {
    "ensaladas": "ensaladas",
    "entrantes": "entrantes",
    "arroces": "arroces",
    "pescado": "pescados-mariscos",
    "carnes": "carnes",
    "postres": "postres",
    "guisos": "segundos-platos", # ? !
    "huevos": "huevos",
    "pasta": "pastas-pizzas"
}

In [44]:
saber_vivir_cat_map = {
    "Arroces": "arroces",
    "Caldos y cremas": "sopas-cremas",
    "Carnes": "carnes",
    "Celebraciones": "otros", # ? !
    "Ensaladas y verduras": "ensaladas",
    "Legumbres": "segundos-platos",
    "Pastas y pizzas": "pastas-pizzas",
    "Pasteles salados": "otros", # ? !
    "Pescados y Mariscos": "pescados-mariscos",
    "Postres y dulces": "postres"
}

In [45]:
el_mundo_cat_map = {
    "postres": 'postres',            
    "pasta": "pastas-pizzas",
    "arroz": "arroces",   
    "pollo": "segundos-platos",
    "sopas-caldos": "sopas-cremas",            
    "bizcochos": "postres",
    "carne": "carnes",                   
    "guisos": "segundos-platos",
    "legumbres": "segundos-platos", 
    "verduras": "segundos-platos", # ? !
    "salsas": "otros",   # ? !        
    "masas-panes": "panes",
    "cremas": "sopas-cremas",
    "patatas-huevos-tortillas": "huevos",
    "aperitivos-faciles": "entrantes",          
    "pescados": "pescados-mariscos",
    "recetas-mariscos": "pescados-mariscos",          
    "vegetarianas": "otros", # ? !
    "sanas-faciles": "otros",  # ? !            
    "orientales": "otros" # ? !
}

In [46]:
(
    all([cat in definitive_categories for cat in hogar_mania_cat_map.values()]) &
    all([cat in definitive_categories for cat in antena3_cat_map.values()]) &
    all([cat in definitive_categories for cat in saber_vivir_cat_map.values()]) &
    all([cat in definitive_categories for cat in el_mundo_cat_map.values()])
)

True

In [47]:
df_hogar_mania['categoria'] = df_hogar_mania['categoria'].map(hogar_mania_cat_map)
df_hogar_mania['categoria'].unique()

array(['postres', 'entrantes', 'ensaladas', 'arroces', 'pastas-pizzas',
       'sopas-cremas', 'segundos-platos', 'carnes', 'pescados-mariscos',
       'huevos', 'desayunos', 'panes', 'bebidas'], dtype=object)

In [48]:
df_antena3['categoria'] = df_antena3['categoria'].map(antena3_cat_map)
df_antena3['categoria'].unique()

array(['ensaladas', 'entrantes', 'arroces', 'pescados-mariscos', 'carnes',
       'postres', 'segundos-platos', 'huevos', 'pastas-pizzas'],
      dtype=object)

In [49]:
df_saber_vivir['categoria'] = df_saber_vivir['categoria'].map(saber_vivir_cat_map)
df_saber_vivir['categoria'].unique()

array(['sopas-cremas', 'postres', 'otros', 'pastas-pizzas',
       'pescados-mariscos', 'ensaladas', 'segundos-platos', 'carnes',
       'arroces'], dtype=object)

In [50]:
df_el_mundo['categoria'] = df_el_mundo['categoria'].map(el_mundo_cat_map)
df_el_mundo['categoria'].unique()

array(['postres', 'pastas-pizzas', 'arroces', 'segundos-platos',
       'sopas-cremas', 'carnes', 'otros', 'panes', 'huevos', 'entrantes',
       'pescados-mariscos'], dtype=object)

In [51]:
list_dataframes = [df_hogar_mania, df_antena3, df_saber_vivir, df_el_mundo]
df_recipes = pd.concat(list_dataframes)
df_recipes.shape

(9487, 8)

In [52]:
df_recipes['categoria'].value_counts()

categoria
ensaladas            1910
carnes               1647
postres              1560
pescados-mariscos    1482
entrantes             572
pastas-pizzas         517
segundos-platos       457
sopas-cremas          408
arroces               384
huevos                318
otros                 100
panes                  74
bebidas                56
desayunos               2
Name: count, dtype: int64

El desbalance de categorías es más que evidente, por lo que se va a prescindir de algunas de ellas porque los registros que existen son muy pocos.

In [53]:
df_recipes = df_recipes[~df_recipes['categoria'].isin(['otros','panes', 'bebidas', 'desayunos'])]
df_recipes.shape

(9255, 8)

Finalmente se exportan los datos limpios al bucket para luego utilizarse en el entrenamiento de modelos

In [54]:
wr.s3.to_csv(
    df=df_recipes,
    path="s3://recipes-data-models-sagemaker-bucket/data/cleaned_recipes.csv",
    index=False
)

{'paths': ['s3://recipes-data-models-sagemaker-bucket/data/cleaned_recipes.csv'],
 'partitions_values': {}}